In [1]:

import argparse
import copy
import os
from enum import Enum
from typing import Dict, List, Union

import numpy as np
import pandas as pd
import torch
from config.configuration import Config
from data.dataloader import GeneralTrainerDataLoader
from data.dataset import GeneralDataset
from data.interaction import Interaction
from data.utils import data_reparation
from models import NeuMF
from root import DATASET_DIR, ROOT_DIR, absolute
from torch.nn.utils import rnn as rnn_utils
from torch.utils.data import Dataset as TorchDataset
from trainer import Trainer
from utils.logger import init_logger

### TODO
- 添加验证集
- NGCF
- LightGCN
- 所有代码整体过一遍
- tensorboard
- 训练参数整理
- checkpoint逻辑
- 日志 ✅

In [3]:

parser = argparse.ArgumentParser()
parser.add_argument(
    "--dataset", "-d", type=str, default="wsdream-rt", help="name of datasets"
)

args, _ = parser.parse_known_args()

config = Config(model="NeuMF", dataset=args.dataset)


init_logger(config)

dataset = GeneralDataset(config)
train_data, test_data = data_reparation(config, dataset)
model = NeuMF(config, dataset)
trainer = Trainer(config, model)

trainer.fit(train_data, test_data, saved=False, show_progress=True)


Train     0:   0%|                                                           | 0/49 [00:00<?, ?it/s]:  18%|█████████▎                                         | 9/49 [00:00<00:00, 87.18it/s]:  39%|███████████████████▍                              | 19/49 [00:00<00:00, 90.76it/s]:  59%|█████████████████████████████▌                    | 29/49 [00:00<00:00, 94.44it/s]:  80%|███████████████████████████████████████▊          | 39/49 [00:00<00:00, 96.13it/s]: 100%|██████████████████████████████████████████████████| 49/49 [00:00<00:00, 96.06it/s]
23-09-07 03:12:32 INFO  epoch 0 training [time: 0.51s, train loss: 38.6221]
Evaluate   :   0%|                                                          | 0/458 [00:00<?, ?it/s]:  20%|█████████▋                                      | 92/458 [00:00<00:00, 919.71it/s]:  40%|██████████████████▉                            | 184/458 [00:00<00:00, 778.77it/s]:  58%|███████████████████████████                    | 264/458 [00:00<00:00, 651.97it/s]:  72%|████

KeyboardInterrupt: 